In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from fuzzywuzzy import process



In [ ]:
df_yield_raw=pd.read_excel('/Users/Roy/Desktop/PaletteSkills/Stream-3__AI_ML_Doc/Python_Class_Practice/data/rm_mb_yield_2004_2023_2.xlsx')
gdf_rm=gpd.read_file('/Users/Roy/Desktop/PaletteSkills/Stream-3__AI_ML_Doc/Python_Class_Practice/data/MUNICIPALITY.geojson', engine="pyogrio")

# Dropping Unncesseary Columns
df_yield_raw_1=df_yield_raw.drop(columns=['Variety', 'Yield/acre(Metric)', 'Yield/acre(Metric).1', 'Yield/acre(Imperial).1' ])

# Creating new clean yield column
df_yield_raw_1['Yield']=df_yield_raw_1['Yield/acre(Imperial)'].str.replace(' Bushels','')

# Dropping old yield column
df_yield_raw_2=df_yield_raw_1.drop(columns='Yield/acre(Imperial)')

# Removing observations that have not enough values
df_yield_raw_3=df_yield_raw_2.loc[df_yield_raw_2['Farms']!='Below']



# Changing column names
df_yield_raw_3=df_yield_raw_3.rename(columns={'Risk Area / R.M.': 'RM'})

gdf_rm=gdf_rm.rename(columns={'MUNI_LIST_NAME': 'RM'})

print(gdf_rm)



# # Replacing , signs Yield and Acres Columns
df_yield_raw_3['Yield']=df_yield_raw_3['Yield'].astype(str).str.replace(',','')
df_yield_raw_3['Acres']=df_yield_raw_3['Acres'].astype(str).str.replace(',','')

df_yield_raw_3['Yield'] = pd.to_numeric(df_yield_raw_3['Yield'], errors='coerce')
df_yield_raw_3['Acres'] = pd.to_numeric(df_yield_raw_3['Acres'], errors='coerce')


df_yield_raw_3['Yield']=df_yield_raw_3['Yield'].astype('Float64')
df_yield_raw_3['Acres']=df_yield_raw_3['Acres'].astype('Float64')
df_yield_raw_3['Farms']=df_yield_raw_3['Farms'].astype(int)

df_yield_raw_3['Crop']=df_yield_raw_3['Crop'].astype('string')
df_yield_raw_3['RM']=df_yield_raw_3['RM'].astype('string')
df_yield_raw_3.info()

In [67]:

def match_rms(rm, choices, limit=2):
    """Fuzzy match an RM name to the closest ones from a list of choices."""
    results = process.extract(rm, choices, limit=limit)
    return results[0]  # Return the best match

# List of RMs in df_yield_raw_3 to match against
yield_rms = df_yield_raw_3['RM'].unique()

# Apply fuzzy matching for RMs in gdf_rm that are not found in df_yield_raw_3
gdf_rm['Matched_RM'] = gdf_rm['RM'].apply(lambda x: match_rms(x, yield_rms))

# Now check the matched results and scores
gdf_rm['Match_Score'] = gdf_rm['Matched_RM'].apply(lambda x: x[1])  # Get the match score
gdf_rm['Matched_RM'] = gdf_rm['Matched_RM'].apply(lambda x: x[0])   # Get the actual matched RM name

# Filter out low-confidence matches if needed (e.g., below 80% similarity)
gdf_rm_filtered = gdf_rm[gdf_rm['Match_Score'] == 100]

gdf_rm_filtered1 = gdf_rm_filtered[gdf_rm_filtered['Match_Score'] < 100]

print(len(df_yield_raw_3))
print(len(gdf_rm_filtered))

print((df_yield_raw_3).columns)
print((gdf_rm_filtered).columns)

# Merge using the fuzzy matched RMs
merged_df_fuzz = pd.merge(gdf_rm_filtered, df_yield_raw_3, left_on='Matched_RM', right_on='RM', how='left')

print((merged_df_fuzz).columns)

gdf_rm_yield=merged_df_fuzz[[ 'Matched_RM', 'Match_Score',
       'Year', 'Crop', 'Farms', 'Acres', 'Yield','geometry']]

print(len(gdf_rm_yield))



33975
93
Index(['Year', 'RM', 'Crop', 'Farms', 'Acres', 'Yield'], dtype='object')
Index(['OBJECTID', 'MUNI_NO', 'MUNI_NAME', 'MUNI_TYPE', 'RM',
       'MUNI_LIST_NAME_WITH_TYPE', 'geometry', 'Matched_RM', 'Match_Score'],
      dtype='object')
Index(['OBJECTID', 'MUNI_NO', 'MUNI_NAME', 'MUNI_TYPE', 'RM_x',
       'MUNI_LIST_NAME_WITH_TYPE', 'geometry', 'Matched_RM', 'Match_Score',
       'Year', 'RM_y', 'Crop', 'Farms', 'Acres', 'Yield'],
      dtype='object')
33720
